In [1]:
from gensim.models.doc2vec import  Doc2Vec, TaggedDocument
from nltk import word_tokenize
import pandas as pd  
import string 

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [15]:
input_data7 = pd.read_csv('input_data7.csv', header = 0,
                          usecols= ['patent_id', 'combined'],
                          dtype = {'patent_id':object},
                          encoding = 'ISO-8859-1')

In [38]:
my_columns = input_data7['patent_id'].tolist() # create patent list
corpus = input_data7['combined'].tolist()
patent_numbers = '|'.join(my_columns)

In [5]:
#preprocessing 


patent_terms = pd.read_csv('patent_terms.csv', encoding = 'ISO-8859-1')
patent_terms = patent_terms.rename(columns={0:'terms'})
patent_terms_append = ['claim','claimed','according','selected','comprising','consisting','means','le','wt','group','thereof','weight']
patent_terms = patent_terms.append(pd.DataFrame(patent_terms_append, columns=['terms']), ignore_index=True)
patent_terms = patent_terms.drop_duplicates()
patent_terms.to_csv('patent_terms.csv', index=False)

jargon = set(patent_terms.ix[:, 0])  # build a "patent jargon" lexical
''' jargon removes word "containing", we want to weigh lignosulfonate which follows!'''

## Punctuation
# nltk.download('punkt')
punctuations = set(string.punctuation)
punctuations.remove('-') # maybe remove hyphens
#punctuations.remove('/')

from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
word_len = 2
def clean(doc):
    number_free = ''.join([c for c in doc if c not in "1234567890"])
    words = [word.strip(string.punctuation) for word in number_free.split(" ")]
    filtered = [f for f in words if f and f.lower() not in stop_words]
    undo = "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in filtered]).strip()
    punc_free = ''.join(ch for ch in undo if ch not in punctuations)
    smallword_free = ' '.join([w for w in punc_free.split() if len(w) > word_len])
    lemmatized = " ".join(lemma.lemmatize(word) for word in smallword_free.split())
    jargon_free = " " .join([j for j in lemmatized.lower().split() if j not in jargon])
    return jargon_free

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # This is added back by InteractiveShellApp.init_path()


In [39]:
corpus_clean = [clean(doc) for doc in corpus] # list of sentance strings
#corpus_tokenize = [clean(doc).split() for doc in corpus]  # list of string words

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # This is added back by InteractiveShellApp.init_path()


In [40]:
corpus_clean[0]

'plywood adhesive phenolic plywood adhesive containing lignosulfonates trialkyl phosphate alkyltituents carbon atom use curtain coater described process applying phenol-formaldehyde resin adhesive containing amylaceous proteinaceous material plywood veneer using curtain coater improvement comprises adding least percent based upon weight phenol-formaldehyde resin said adhesive sodium base lignosulfonate percent based upon weight phenol-formaldehyde resin trialkyl phosphate alkyltituents carbon atomsa process according claim wherein trialkyl phosphate tributyl phosphatea process according claim wherein sodium base lignosulfonate added amount weight percenta process according claim wherein trialkyl phosphate tributyl phosphate added amount percenta process according claim wherein sodium lignosulfonate added phenol formaldehyde mixture condensation phenol formaldehyde preparation phenol-formaldehyde resina process according claim wherein lignosulfonate added amount weight percent trialkyl 

In [6]:
data = data_df['combined'].tolist()

In [41]:
tagged_data = [TaggedDocument(words= word_tokenize(_d.lower()), tags = [str(i)])
              for i, _d in enumerate(corpus_clean)]

#Need to figure out how to make the patent ID the document tag
# tagged_data = [TaggedDocument(words= word_tokenize(_d.lower()), tags = [str(i)])
#               for i, _d in enumerate(data)]








In [42]:
len(tagged_data)

5651

In [48]:
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(size= vec_size,
               alpha = alpha,
                min_alpha = 0.0025,
                min_count = 1, 
                dm = 1
               )

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("patent_test_d2v.model")
print("Model Saved")

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3


KeyboardInterrupt: 

In [2]:
modell = Doc2Vec.load('patentd2v_model')

In [4]:
test_patent = ('''A lock for a height-adjustable crib or playpen requires two distinct manual operations in order to release 
the lock on each crib leg. Each lock includes a rigid metal bracket which engages around the extensible part o
f the leg and is pivotally connected to a crib corner post. The bracket has a nose which projects into one of a 
series of openings in the extensible part of the leg then the bracket is swung against the leg part to prevent
movement of the leg part relative to the post. The upper edge of the bracket is slotted to receive a latch pivotally 
connected to the corner post just above the bracket. The latch drops into the slot when the nose is engaged in one of the openings 
so that the bracket cannot be moved.''')

In [7]:
test_clean = word_tokenize(clean(test_patent))

In [8]:
test_clean

['lock',
 'height-adjustable',
 'crib',
 'playpen',
 'requires',
 'two',
 'distinct',
 'manual',
 'operation',
 'order',
 'release',
 'the',
 'lock',
 'crib',
 'leg',
 'lock',
 'includes',
 'rigid',
 'metal',
 'bracket',
 'engages',
 'around',
 'extensible',
 'part',
 'leg',
 'pivotally',
 'connected',
 'crib',
 'corner',
 'post',
 'bracket',
 'nose',
 'project',
 'one',
 'series',
 'opening',
 'extensible',
 'part',
 'leg',
 'bracket',
 'swung',
 'leg',
 'part',
 'prevent',
 'movement',
 'leg',
 'part',
 'relative',
 'post',
 'upper',
 'edge',
 'bracket',
 'slotted',
 'receive',
 'latch',
 'pivotally',
 'connected',
 'corner',
 'post',
 'bracket',
 'latch',
 'drop',
 'slot',
 'nose',
 'engaged',
 'one',
 'opening',
 'bracket',
 'can',
 'not',
 'moved']

In [11]:
v1 = modell.infer_vector(test_clean)
print("V1_infer", v1)




V1_infer [ -3.96660306e-02   1.66998692e-02  -1.08166777e-01   1.12937242e-01
  -2.81128854e-01   9.43073258e-02  -1.01115238e-02  -4.07120809e-02
  -1.50741175e-01   1.29770204e-01  -3.98794152e-02  -1.27012193e-01
   6.49911165e-02   2.02479929e-01   1.73687577e-01  -2.29176372e-01
  -5.26362546e-02  -5.44692874e-02   9.27926451e-02  -7.23105893e-02
  -3.69648486e-02  -6.89362809e-02  -1.31452113e-01   2.25270331e-01
  -2.27699608e-01  -7.20152184e-02   1.16083147e-02   2.78145298e-02
  -2.55966578e-02  -5.82129240e-01   3.23467821e-01  -2.22025484e-01
  -1.67734534e-01  -1.48489133e-01  -1.58060744e-01  -6.17329404e-02
   2.28977367e-01   3.48583795e-02  -1.73888937e-01   2.64412940e-01
   8.88824016e-02  -1.77263990e-01  -3.02927122e-02   2.53726840e-01
   1.19511388e-01   2.06096336e-01  -7.25827888e-02   1.84041813e-01
   2.40855217e-01  -1.01044672e-02   3.53014395e-02  -1.59615144e-01
   3.10161769e-01  -3.67261246e-02  -9.50097218e-02  -9.54128355e-02
  -2.27135420e-02   2.102

In [13]:
# to find most similar doc using tags
similar_doc = modell.docvecs.most_similar('1')
print(similar_doc)

[('306', 0.4959915578365326), ('123', 0.4942142367362976), ('76', 0.4361250698566437), ('41', 0.4006164073944092), ('222', 0.3942888379096985), ('4931', 0.3793490529060364), ('230', 0.3686369061470032), ('5500', 0.3670188784599304), ('5153', 0.36132562160491943), ('4270', 0.35854288935661316)]


In [18]:
input_data7['combined'].iloc[1]

'Apparatus for heating a window A grid of heating conductors for defrosting applied to the window of a vehicle is connected to a source of electrical energy through a relay-operated switch. This switch is closed either in response to a drop in the resistance between two conductors or electrodes mounted in spaced relation on an insulatingort such as the window (the drop occurring by accumulation of moisture on theort and between the conductors) or in response to a drop in temperature at or near the window. For this purpose the resistance between the two conductors over the surface of theort may be paralleled by a thermostatic switch which closes with drop in temperature, both this resistance and the thermostatic switch being in the input to a trigger circuit controlling the relay to energize it in response to a decline in resistance at that input. Alternatively the trigger circuit may include thermistors which sense the drop in temperature and set the trigger circuit to the condition wh

In [20]:
input_data7['combined'].iloc[306]

'Electrically heated vehicle window A humidity-responsive heating circuit for a window of an automotive vehicle includes a grid of heating conductors on the window and two conductors closely approaching each other on the surface of the window such that the resistance between them changes, upon the appearance of humidity bridging the space between them, sufficiently to provide a signal. All of the conductors are composed of the same paste having the same properties and are applied as a single baked, silk screen printed pattern. The circuit may further include a switch to apply a voltage between those two conductors, amplifying means responsive to change of resistance between them, and a relay controlled by the amplifying means for connection of the heating grid to a source of voltage. Apparatus for heating a glass window comprising, on the inside surface of a window, a plurality of spaced electrical resistance heating conductors connected between bus bars and at least one additional con

In [21]:
input_data7['combined'].iloc[123]

'Control system for vehicle window heater Apparatus for controllably heating a vehicle window having a heating grid is described including an oscillator and a signal detector, with a signal circuit coupling the oscillator and the signal detector having a DC voltage component at one or more points thereof. One or more humidity detectors are coupled to the signal circuit to control the amplitude of the oscillationslied from the oscillator to the signal detector, advantageously by voltage divider action, and capacitor means provides DC isolation between the humidity detector(s) and any DC voltage component in the signal circuit, and also between humidity detectors when a plurality are used. Advantageously the humidity detector(s) are directly connected to the heating grid, and varying DC voltage levels at the point(s) of connection to the grid are isolated by the capacitor means. Particular arrangements of the humidity detectors on the window and connections to the heating grid are descri